In [1]:
import numpy as np
import torch.nn as nn
import json
import matplotlib
import os
import random
import torch

from collections import Counter
from itertools import islice
from sklearn.utils import shuffle
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset

from models.Encoder import *
from models.Decoder import *
from models.utils import *
from utils import *

# set fixed random seed to reproduce results
np.random.seed(42)
random.seed(42)

%matplotlib inline

GPU is available
GPU is available


# Experiment 1b

In [2]:
# define experiment
exp='/exp_1'

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds_train, act_vocab, w2i_acts, i2w_acts, acts_train = load_dataset(exp=exp, split='/train')
_, _, _, cmds_test, _, _, _, acts_test = load_dataset(exp=exp, split='/test')

In [5]:
## create input and output language pairs ##

# training
train_cmd_act_pairs = create_pairs(cmds_train, acts_train)
print("Number of train source-target pairs: {}".format(len(train_cmd_act_pairs)))

# testing
test_cmd_act_pairs = create_pairs(cmds_test, acts_test)
print("Number of test source-target pairs: {}".format(len(test_cmd_act_pairs)))

Number of train source-target pairs: 16728
Number of test source-target pairs: 4182


In [6]:
# show random train command-action pair
random_pair = random.choice(train_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['look', 'left', 'thrice', 'after', 'turn', 'opposite', 'left']
Action: ['I_TURN_LEFT', 'I_TURN_LEFT', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK']


In [7]:
# show random test command-action pair
random_pair = random.choice(test_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['run', 'right', 'thrice', 'after', 'jump', 'right']
Action: ['I_TURN_RIGHT', 'I_JUMP', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN', 'I_TURN_RIGHT', 'I_RUN']


In [8]:
cmd_act_pair = pairs2idx(random_pair, w2i_cmds, w2i_acts)
print("Command sequence: {}".format(cmd_act_pair[0]))
print("Action sequence: {}".format(cmd_act_pair[1]))

Command sequence: tensor([14,  4,  7, 10, 12,  4,  2])
Action sequence: tensor([1, 4, 6, 4, 7, 4, 7, 4, 7, 2])


In [9]:
## Hyperparameters for training ##

# source language (i.e., commands) vocabulary size |V_source|
in_size = len(w2i_cmds)

# target language (i.e., actions) vocabulary size |V_target|
out_size = len(w2i_acts)

# size of word embeddings
emb_size = 20

# size of hidden units
hidden_size = 50 #100

# number of layers
n_layers = 2

# number of epochs
n_epochs = 2 # 2-4 epochs (100.000 iterations each) seem to be sufficient to learn the mapping

In [10]:
## Instantiate models ##

encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers)
decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers)

In [11]:
# move models to GPU, if GPU is available (for faster computation)
encoder.cuda()
decoder.cuda()

DecoderLSTM(
  (embedding): Embedding(10, 20)
  (lstm): LSTM(20, 50, num_layers=2, dropout=0.5)
  (linear): Linear(in_features=50, out_features=10, bias=True)
)

## Training (1a)

In [12]:
# train_losses, train_accs, encoder, decoder = train(train_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder, n_epochs)

## Testing (1a)

In [13]:
# test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)

## Train and test over different numbers of distinct source-target pairs (1b)

In [ ]:
## experiment 1b train and test loop over pre-defined ratios ##

ratios = np.array([0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
accs_per_ratio = {}

for ratio in ratios:
    train_samples = sample_distinct_pairs(train_cmd_act_pairs, ratio)
    print("-------------------------------------------------")
    print("Current percentage of total commands used for training: {}%".format(ratio*100))
    print("Number of distinct examples shown during training: {}".format(len(train_samples)))
    print("-------------------------------------------------")
    # instantiate new encoder and decoder for each ratio
    encoder = EncoderLSTM(in_size, emb_size, hidden_size, n_layers)
    decoder = DecoderLSTM(emb_size, hidden_size, out_size, n_layers)
    encoder.cuda()
    decoder.cuda()
    g
    train_losses, train_accs, encoder, decoder = train(train_samples, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder, n_epochs)
    test_acc = test(test_cmd_act_pairs, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder)
    accs_per_ratio[ratio] = test_acc

-------------------------------------------------
Current percentage of total commands used for training: 1.0%
Number of distinct examples shown during training: 167


Epoch:   0%|                                                                                     | 0/2 [00:00<?, ?it/s]

Loss: 0.24522526264190675
Acc: 0.0837916208379162

Command: walk left thrice after jump left <EOS>
True action: I_TURN_LEFT I_JUMP I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK <EOS>
Pred action: I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT

True sent length: 9
Pred sent length: 9

Loss: 0.0691698881296011
Acc: 0.28583570821458926

Command: run opposite right thrice and walk twice <EOS>
True action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_WALK I_WALK <EOS>
Pred action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_WALK I_WALK <EOS>

True sent length: 12
Pred sent length: 12

Loss: 0.039535204569498696
Acc: 0.4392853571547615

Command: jump right twice and walk opposite left twice <EOS>
True action: I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK <EOS>
Pred action: I_TURN_

Epoch:  50%|███████████████████████████████████▌                                   | 1/2 [1:45:37<1:45:37, 6337.11s/it]

Loss: 0.00046238899230957033
Acc: 0.9365063493650635

Command: walk left thrice after jump left <EOS>
True action: I_TURN_LEFT I_JUMP I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK <EOS>
Pred action: I_TURN_LEFT I_JUMP I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK <EOS>

True sent length: 9
Pred sent length: 9

Loss: 0.0007960246159480169
Acc: 0.9452027398630068

Command: run opposite right thrice and walk twice <EOS>
True action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_WALK I_WALK <EOS>
Pred action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_WALK I_WALK <EOS>

True sent length: 12
Pred sent length: 12

Loss: 0.0010148684183756511
Acc: 0.9492683577214093

Command: jump right twice and walk opposite left twice <EOS>
True action: I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK <EOS>
Pred action: I_TURN_

...

Loss: 0.0026381492614746095
Acc: 0.8789121087891211

Command: walk right thrice and walk twice <EOS>
True action: I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_WALK I_WALK <EOS>
Pred action: I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_WALK I_WALK <EOS>

True sent length: 9
Pred sent length: 9

Loss: 0.0038605928421020508
Acc: 0.8938553072346382

Command: run left and run right twice <EOS>
True action: I_TURN_LEFT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN <EOS>
Pred action: I_TURN_LEFT I_RUN I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN <EOS>

True sent length: 7
Pred sent length: 7

Loss: 0.0009091741898480584
Acc: 0.9009366354454852

Command: jump around left thrice and look around left <EOS>
True action: I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_LOOK I_TURN_LE

Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [3:33:49<00:00, 6414.58s/it]


Test acc: 0.2097902097902098

Command: walk opposite left twice after walk opposite left thrice <EOS>
True action: I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK <EOS>
Pred action: I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT <EOS>

True sent length: 16
Pred sent length: 15

Test acc: 0.20389805097451275

Command: turn right twice after turn opposite left thrice <EOS>
True action: I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_RIGHT I_TURN_RIGHT <EOS>
Pred action: I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT <EOS>

True sent length: 9
Pred sent length: 11

Test acc: 0.20226591136287905

Command: run around right after look opposite left <EOS>
True action: I_TURN_LE

Epoch:   0%|                                                                                     | 0/2 [00:00<?, ?it/s]

Loss: 0.29207386690027576
Acc: 0.0463953604639536

Command: run opposite right thrice and run left thrice <EOS>
True action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN <EOS>
Pred action: I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_LEFT I_TURN_RIGHT I_RUN I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_RUN I_TURN_LEFT I_RUN <EOS>

True sent length: 16
Pred sent length: 16

Loss: 0.20264506340026855
Acc: 0.17639118044097796

Command: look right thrice and turn around left twice <EOS>
True action: I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT <EOS>
Pred action: I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT

True sent length: 15
Pred sent 

In [ ]:
# save results in .json file
with open('./results/experiment_1b', 'w') as json_file:
      json.dump(accs_per_ratio, json_file)